# Data loading

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import re
import numpy as np
import os
import warnings

# For progress bar
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable, desc=""):
        return iterable

# ---------------------------------------------------------------------
# 1) Configuration / Setup
# ---------------------------------------------------------------------

# 1a) Suppress warnings
warnings.filterwarnings(
    "ignore",
    message="The number of bins estimated may be suboptimal."
)
warnings.filterwarnings(
    "ignore",
    category=DeprecationWarning,
    message="DataFrameGroupBy.apply operated on the grouping columns"
)

# 1b) Instrument, file patterns, output directory
instr = 'maxi'
os.makedirs(f"plots/{instr}", exist_ok=True)

file_pattern_full = (
    f'/disk/data/youssef/scripts/xrb-population/results_latest/{instr}_results/*full*.csv'
)
file_pattern_agg  = (
    f'/disk/data/youssef/scripts/xrb-population/results_latest/{instr}_results/*.csv'
)

# Identify the files
csv_files_full = [f for f in glob(file_pattern_full)]
csv_files_agg  = [f for f in glob(file_pattern_agg) if "full" not in f]

# The columns we ultimately want in the aggregated file
all_needed_cols = [
    "red_chi_squared", "gamma", "temp", "power_norm_fit",
    "disk_norm_fit", "total_flux_median", "d_fit_median",
    "d_fit_peak", "peak_flux", "d_fit_weighted_median",
    "error_d_median", "frac_uncert_median", "error_d_peak",
    "frac_uncert_peak"
]

# This is the central column (distance statistic) used for filtering
central_value = "d_fit_weighted_median"

# ---------------------------------------------------------------------
# 2) Global Data Structures
# ---------------------------------------------------------------------
# Same structures from your original script
distances       = []
na_distances    = []
tight_distances = []
na_nH           = []
all_data        = []
na_d_fit_files  = []
na_d_fit_params = []
all_data_params = []

# ---------------------------------------------------------------------
# 3) Helper Functions
# ---------------------------------------------------------------------
def extract_parameters(filename):
    """
    Extract parameter values (g, T, a, m, i, r, e) from the filename 
    using regular expressions. Adjust the pattern to match your filenames.
    """
    pattern = (
        r'table_g(?P<g>\d+\.\d+)_T(?P<T>\d+\.\d+)_a(?P<a>\d+\.\d+)_'
        r'm(?P<m>\d+\.\d+)_i(?P<i>\d+\.\d+)_r(?P<r>\d+\.\d+)_e(?P<e>\d+\.\d+)'
    )
    match = re.search(pattern, filename)
    if match:
        return {key: float(value) for key, value in match.groupdict().items()}
    else:
        return None

def calc_MAE(scale, l_err, u_err):
    """
    Calculate Mean Absolute Error (MAE) and safeguard reciprocal weight calculation.

    Parameters:
        scale (float): The scaling factor for error normalization.
        l_err (numpy.ndarray): Lower error values.
        u_err (numpy.ndarray): Upper error values.

    Returns:
        numpy.ndarray: The computed MAE array with safeguards.
    """
    # Check that l_err and u_err are of the same length
    if len(l_err) != len(u_err):
        raise ValueError("`l_err` and `u_err` must have the same length.")
    
    # Ensure no negative or zero error intervals
    error_diff = u_err - l_err
    if np.any(error_diff <= 0):
        warnings.warn(
            f"Non-positive error intervals detected: {error_diff[error_diff <= 0]} ... {u_err[error_diff <= 0]} ... {l_err[error_diff <= 0]} will take absolute of the difference")
    
    # Calculate MAE
    mae = (error_diff / 2) / scale

    # Check for invalid or NaN results
    if np.any(np.isnan(mae)):
        raise ValueError("MAE calculation resulted in NaN values.")
    
    return np.abs(mae)


def weighted_median(values, weights):
    """
    Compute the weighted median of a 1D NumPy array `values` 
    with positive `weights`.

    Parameters:
        values (numpy.ndarray): Array of data values.
        weights (numpy.ndarray): Array of weights corresponding to `values`.

    Returns:
        float or None: The weighted median, or None if inputs are invalid.
    """
    # Input validation
    if not isinstance(values, np.ndarray) or not isinstance(weights, np.ndarray):
        raise TypeError("Both `values` and `weights` must be NumPy arrays.")
    if values.ndim != 1 or weights.ndim != 1:
        raise ValueError("Both `values` and `weights` must be 1D arrays.")
    if len(values) == 0:
        raise ValueError("`values` array cannot be empty.")
    if len(values) != len(weights):
        raise ValueError("`values` and `weights` must have the same length.")
    if np.any(weights < 0):
        raise ValueError(f"`weights` must contain only non-negative values. {weights[weights < 0]}")
    if np.any(np.isnan(weights)):
        raise ValueError("`weights` cannot contain NaN values.")
    if np.sum(weights) == 0:
        raise ValueError("Sum of `weights` must be greater than zero.")

    # Sort values and weights by `values`
    sort_idx = np.argsort(values)
    sorted_values = values[sort_idx]
    sorted_weights = weights[sort_idx]

    # Compute cumulative weights
    cumulative_weights = np.cumsum(sorted_weights)
    total_weight = cumulative_weights[-1]
    midpoint = 0.5 * total_weight

    # Find the index where cumulative weight crosses the midpoint
    median_idx = np.searchsorted(cumulative_weights, midpoint, side='right')

    return sorted_values[median_idx]

def find_peak(array):
    """
    Find the peak (mode) of the distribution by computing a histogram.
    The bin size uses 'stone' by default. Adjust if needed.
    """
    if len(array) == 0:
        return None
    counts, bins = np.histogram(array, bins='stone')
    peak_idx = np.argmax(counts)
    return 0.5 * (bins[peak_idx] + bins[peak_idx + 1])

# ---------------------------------------------------------------------
# 4) Function to compute ALL columns
# ---------------------------------------------------------------------
def compute_all_stats(df):
    """
    Compute all columns for each group (nH, d).
    """
    nH_val = df['nH'].iloc[0] if 'nH' in df.columns else None
    d_val  = df['d'].iloc[0]  if 'd'  in df.columns else None

    d_fit_peak = find_peak(df['d_fit'].dropna()) if 'd_fit' in df.columns else None
    total_flux_peak = find_peak(df['total_flux'].dropna()) if 'total_flux' in df.columns else None

    if {'d_fit', 'error_d_low', 'error_d_up'}.issubset(df.columns):
        valid_mask = (
            df['d_fit'].notna() &
            df['error_d_low'].notna() &
            df['error_d_up'].notna()
        )
        if valid_mask.any():
            w = 1 / calc_MAE(
                df.loc[valid_mask, 'd_fit'],
                df.loc[valid_mask, 'error_d_low'],
                df.loc[valid_mask, 'error_d_up']
            )
            d_fit_weighted_med = weighted_median(
                df.loc[valid_mask, 'd_fit'].values,
                w.values
            )
        else:
            d_fit_weighted_med = None
    else:
        d_fit_weighted_med = None

    if d_fit_peak is not None and d_val is not None:
        peak_dist_err = d_fit_peak - d_val
        frac_peak_err = peak_dist_err / d_val if d_val != 0 else None
    else:
        peak_dist_err = None
        frac_peak_err = None

    if 'd_fit' in df.columns and df['d_fit'].notna().any() and d_val is not None:
        d_fit_median = df['d_fit'].median()
        error_d_median = d_fit_median - d_val
        frac_uncert_median = (
            error_d_median / d_val 
            if d_val != 0 else None
        )
    else:
        d_fit_median = None
        error_d_median = None
        frac_uncert_median = None

    return pd.Series({
        "nH": nH_val,
        "d": d_val,

        "red_chi_squared": df['red_chi_squared'].median() if 'red_chi_squared' in df.columns else None,
        "gamma": df['gamma'].median() if 'gamma' in df.columns else None,
        "temp": df['temp'].median() if 'temp' in df.columns else None,

        "power_norm_fit": df['power_norm_fit'].median() if 'power_norm_fit' in df.columns else None,
        "disk_norm_fit": df['disk_norm_fit'].median() if 'disk_norm_fit' in df.columns else None,
        "total_flux_median": df['total_flux'].median() if 'total_flux' in df.columns else None,

        "d_fit_median": d_fit_median,
        "d_fit_peak": d_fit_peak,
        "peak_flux": total_flux_peak,
        "d_fit_weighted_median": d_fit_weighted_med,

        "error_d_median": error_d_median,
        "frac_uncert_median": frac_uncert_median,
        "error_d_peak": peak_dist_err,
        "frac_uncert_peak": frac_peak_err
    })

# ---------------------------------------------------------------------
# 5) Function to compute ONLY a subset of columns
# ---------------------------------------------------------------------
def compute_subset_stats(df, needed_cols):
    """
    Compute only the columns in `needed_cols` for each group (nH, d).
    Return them plus (nH, d).
    """
    nH_val = df['nH'].iloc[0] if 'nH' in df.columns else None
    d_val  = df['d'].iloc[0]  if 'd'  in df.columns else None

    result = {
        "nH": nH_val,
        "d": d_val,
    }

    # Example logic for 'd_fit_peak', 'error_d_peak', 'frac_uncert_peak'
    if "d_fit_peak" in needed_cols or "error_d_peak" in needed_cols or "frac_uncert_peak" in needed_cols:
        d_fit_peak = find_peak(df['d_fit'].dropna()) if 'd_fit' in df.columns else None
        if "d_fit_peak" in needed_cols:
            result["d_fit_peak"] = d_fit_peak
        if d_fit_peak is not None and d_val is not None:
            peak_dist_err = d_fit_peak - d_val
            frac_peak_err = peak_dist_err / d_val if d_val != 0 else None
        else:
            peak_dist_err = None
            frac_peak_err = None
        if "error_d_peak" in needed_cols:
            result["error_d_peak"] = peak_dist_err
        if "frac_uncert_peak" in needed_cols:
            result["frac_uncert_peak"] = frac_peak_err

    if "peak_flux" in needed_cols:
        result["peak_flux"] = find_peak(df['total_flux'].dropna()) if 'total_flux' in df.columns else None

    if "d_fit_weighted_median" in needed_cols:
        if {'d_fit', 'error_d_low', 'error_d_up'}.issubset(df.columns):
            valid_mask = (
                df['d_fit'].notna() &
                df['error_d_low'].notna() &
                df['error_d_up'].notna()
            )
            if valid_mask.any():
                w = 1 / calc_MAE(
                    df.loc[valid_mask, 'd_fit'],
                    df.loc[valid_mask, 'error_d_low'],
                    df.loc[valid_mask, 'error_d_up']
                )
                result["d_fit_weighted_median"] = weighted_median(
                    df.loc[valid_mask, 'd_fit'].values,
                    w.values
                )
            else:
                result["d_fit_weighted_median"] = None
        else:
            result["d_fit_weighted_median"] = None

    if ("d_fit_median" in needed_cols or 
        "error_d_median" in needed_cols or 
        "frac_uncert_median" in needed_cols):
        
        if 'd_fit' in df.columns and df['d_fit'].notna().any() and d_val is not None:
            d_fit_median = df['d_fit'].median()
            error_d_median = d_fit_median - d_val
            frac_uncert_median = (error_d_median / d_val) if d_val != 0 else None
        else:
            d_fit_median = None
            error_d_median = None
            frac_uncert_median = None

        if "d_fit_median" in needed_cols:
            result["d_fit_median"] = d_fit_median
        if "error_d_median" in needed_cols:
            result["error_d_median"] = error_d_median
        if "frac_uncert_median" in needed_cols:
            result["frac_uncert_median"] = frac_uncert_median

    if "red_chi_squared" in needed_cols:
        result["red_chi_squared"] = df['red_chi_squared'].median() if 'red_chi_squared' in df.columns else None

    if "gamma" in needed_cols:
        result["gamma"] = df['gamma'].median() if 'gamma' in df.columns else None

    if "temp" in needed_cols:
        result["temp"] = df['temp'].median() if 'temp' in df.columns else None

    if "power_norm_fit" in needed_cols:
        result["power_norm_fit"] = df['power_norm_fit'].median() if 'power_norm_fit' in df.columns else None

    if "disk_norm_fit" in needed_cols:
        result["disk_norm_fit"] = df['disk_norm_fit'].median() if 'disk_norm_fit' in df.columns else None

    if "total_flux_median" in needed_cols:
        result["total_flux_median"] = df['total_flux'].median() if 'total_flux' in df.columns else None

    return pd.Series(result)

# ---------------------------------------------------------------------
# 6) Main Loop: Only Compute Missing Columns, Collect Data Structures
# ---------------------------------------------------------------------
merged_results = []

for file in tqdm(csv_files_full, desc="Processing FULL CSV Files"):
    # 6a) Extract parameters
    params = extract_parameters(file)
    if params is None:
        continue

    base_name_full = os.path.basename(file)
    base_name_no_ext = os.path.splitext(base_name_full)[0].replace("_full", "")

    # 6b) Attempt to find existing aggregated file
    existing_agg = None
    for f_agg in csv_files_agg:
        if base_name_no_ext in f_agg:
            try:
                existing_agg = pd.read_csv(f_agg)
                break
            except Exception as e:
                print(f"Error reading {f_agg}: {e}")
                existing_agg = None

    # 6c) Load the FULL CSV data
    try:
        full_data = pd.read_csv(file)
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue

    # Must have 'nH' and 'd' for grouping
    if 'nH' not in full_data.columns or 'd' not in full_data.columns:
        continue

    # 6d) If no existing aggregated file, compute ALL columns
    if existing_agg is None:
        aggregated_df = (
            full_data
            .groupby(['nH', 'd'], as_index=False)
            .apply(compute_all_stats)
            .reset_index(drop=True)
        )
    else:
        # Identify which columns are missing
        missing_cols = [c for c in all_needed_cols if c not in existing_agg.columns]
        if not missing_cols:
            # Everything is already present
            aggregated_df = existing_agg.copy()
        else:
            # Compute only the missing columns
            partial_agg = (
                full_data
                .groupby(['nH', 'd'], as_index=False)
                .apply(lambda grp: compute_subset_stats(grp, missing_cols))
                .reset_index(drop=True)
            )
            # Merge partial_agg with existing_agg on (nH, d)
            aggregated_df = pd.merge(
                existing_agg,
                partial_agg,
                on=["nH", "d"],
                how="outer",
                suffixes=("_old", "")
            )
            # Fill missing columns from partial_agg
            for col in missing_cols:
                old_col_name = col + "_old"
                new_col_name = col
                if old_col_name in aggregated_df.columns and new_col_name in aggregated_df.columns:
                    # Fill the old column's NaNs with new column's values
                    mask_na = aggregated_df[old_col_name].isna()
                    aggregated_df.loc[mask_na, old_col_name] = aggregated_df.loc[mask_na, new_col_name]
                    # Drop new col, rename old -> col
                    aggregated_df.drop(columns=[new_col_name], inplace=True)
                    aggregated_df.rename(columns={old_col_name: col}, inplace=True)
                elif old_col_name in aggregated_df.columns and new_col_name not in aggregated_df.columns:
                    # The new aggregator didn't produce that col, rename old -> new
                    aggregated_df.rename(columns={old_col_name: col}, inplace=True)

            # Remove leftover columns with _old suffix
            leftover_old = [c for c in aggregated_df.columns if c.endswith("_old")]
            aggregated_df.drop(columns=leftover_old, inplace=True)

    # 6e) Collect data for distances, na_distances, etc.
    # Filter rows where `central_value` is present
    valid_data = aggregated_df[aggregated_df[central_value].notna()]
    distances.extend(valid_data['d'].values)

    filtered_data = valid_data[(np.abs(valid_data['d']-valid_data[central_value])/valid_data['d']) <= 0.5]
    tight_distances.extend(filtered_data['d'].values)

    # Rows where `central_value` is NaN
    na_data = aggregated_df[aggregated_df[central_value].isna()]
    na_distances.extend(na_data['d'].values)
    na_nH.extend(na_data['nH'].values)

    # Keep the entire aggregated data in `all_data`
    all_data.append(aggregated_df)
    all_data_params.append(params)

    # Check if 'd' == 1 for missing `central_value`
    selected_data = aggregated_df[aggregated_df['d'] == 1]
    if aggregated_df[central_value].isna().any():
        na_d_fit_files.append(os.path.basename(file))

    na_d_fit_params.extend([params]*len(na_data['d'].values))

    merged_results.append(aggregated_df)

# ---------------------------------------------------------------------
# 7) Combine everything (optional) and finalize
# ---------------------------------------------------------------------
final_merged_df = pd.concat(merged_results, ignore_index=True)

# If you want to save the final result:
# final_merged_df.to_csv("final_merged_aggregated_stats.csv", index=False)

print("Done. Only missing columns were computed for existing aggregated files.")
print(f"Collected {len(distances)} valid distances and {len(na_distances)} missing distances.")


Processing FULL CSV Files:  58%|█████▊    | 839/1458 [01:32<01:31,  6.73it/s]/tmp/ipykernel_755700/3024815655.py:109: UserWarning: Non-positive error intervals detected: 5589   -179.019997
dtype: float64 ... 5589    18.009702
Name: error_d_up, dtype: float64 ... 5589    197.029699
Name: error_d_low, dtype: float64 will take absolute of the difference
  warnings.warn(
Processing FULL CSV Files:  82%|████████▏ | 1189/1458 [02:13<00:32,  8.20it/s]/tmp/ipykernel_755700/3024815655.py:109: UserWarning: Non-positive error intervals detected: 1914   -3.31801
dtype: float64 ... 1914    6.210822
Name: error_d_up, dtype: float64 ... 1914    9.528832
Name: error_d_low, dtype: float64 will take absolute of the difference
  warnings.warn(
Processing FULL CSV Files:  88%|████████▊ | 1277/1458 [02:22<00:17, 10.26it/s]/tmp/ipykernel_755700/3024815655.py:109: UserWarning: Non-positive error intervals detected: 567   -0.912259
dtype: float64 ... 567    1.542494
Name: error_d_up, dtype: float64 ... 567   

Done. Only missing columns were computed for existing aggregated files.
Collected 36221 valid distances and 22099 missing distances.


In [2]:
all_data_with_params = []
for data_df, params in zip(all_data, all_data_params):  # Match each dataset with its parameters
    # Repeat the parameter dictionary for each row in the corresponding data_df
    params_df = pd.DataFrame([params] * len(data_df))
    # Concatenate data and parameters
    data_with_params = pd.concat([data_df.reset_index(drop=True), params_df.reset_index(drop=True)], axis=1)
    all_data_with_params.append(data_with_params)

# Step 2: Concatenate all entries into a single DataFrame
all_data_flat = pd.concat(all_data_with_params, ignore_index=True)
all_data_flat['diff_d'] = all_data_flat['d_fit_weighted_median'] - all_data_flat['d']

all_data_flat['instr']= instr


In [3]:
import sqlalchemy
import pandas as pd

db = sqlalchemy.create_engine('sqlite:///results.db')

try:
    with db.connect() as conn:
        print(f"Opened SQLite database with sqlalchemy version {sqlalchemy.__version__} successfully.")
        xrt_results = pd.read_sql('SELECT * FROM maxi',conn)
except Exception as e:
    print("Failed to open database:", e)

Opened SQLite database with sqlalchemy version 2.0.30 successfully.
